Ideas for the DashBoard
========================

* Track the activity of a user- No graphs needed, can show the data in tabular format
* To track the stats od the overall graph, need graphs, perhaps those being updated in real time

Streaming graph to track the number of tweets with cetain hashag
------------------------------------------------------------------

In [1]:
from __future__ import print_function
import plotly
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
import pandas as pd
import datetime, time, random

In [2]:
plotly.tools.set_credentials_file(username='sainideepak119', api_key='X6vwN8OWBMVmHGZhJecY')

In [3]:
stream_ids = tls.get_credentials_file()['stream_ids']
print(stream_ids)

['hbf4ycoqv5', '16nhy11qxu', '500i1u8gql', 'j11d4bttjl']


In [4]:
stream_id = stream_ids[-1]

stream_1 = go.Stream(
    token=stream_id,
    maxpoints=80,
)

In [5]:
trace1 = go.Scatter(
    x=[],
    y=[],
    mode='lines+markers',
    stream=stream_1
)

data = go.Data([trace1])

In [6]:
layout = go.Layout(title='Time Series')
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='python-streaming')

In [7]:
import os
os.environ['HTTP_PROXY']="http://proxy62.iitd.ac.in:3128"
os.environ['HTTPS_PROXY']="https://proxy62.iitd.ac.in:3128"

In [7]:
pipe = py.Stream(stream_id)
pipe.open()

In [8]:
i = 0
k = 5
time.sleep(5)

while True:
    x = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')
    y = (np.cos(k*i/50.)*np.cos(i/50.)+np.random.randn(1))[0]
    pipe.write(dict(x=x, y=y))
    time.sleep(1)

KeyboardInterrupt: 

In [9]:
pipe.close()

Stream bubble map of locations of tweets
-----------------------------------------

In [10]:
dframe = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_february_us_airport_traffic.csv')
dframe["text"] = "Airport Code "+dframe["iata"]+"<br/> State: "+dframe["state"]
print(len(dframe))

221


In [11]:
stream_ids = tls.get_credentials_file()['stream_ids']
print(stream_ids)
stream_id = stream_ids[-2]

stream_1 = dict(
    token=stream_id,
    maxpoints=10,
)

['hbf4ycoqv5', '16nhy11qxu', '500i1u8gql', 'j11d4bttjl']


In [15]:
data = [dict(
        type='scattergeo',
        lon=[],
        lat=[],
        text = [],
        mode='markers',
        marker=dict(
            size=8,
            opacity=0.8,
            reversescale=True,
            autocolorscale=False,
            line=dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
        ),
        stream=stream_1,
        name="City")]
layout = dict(
        title = 'Random cities',
        showlegend = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )


fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='geo-streaming2')#, auto_open=False, fileopt='extend')

In [13]:
pipe = py.Stream(stream_id)
pipe.open()

In [14]:
time.sleep(5)

while True:
    i = random.randint(0,221)
#     print(i)
    row = dframe.iloc[i]
#     print(row["lat"],row["long"])
    pipe.write(dict(lat=row["lat"], lon=row["long"],text = row["text"],type="scattergeo",))
#                     marker={'size': 5, 'sizemode': 'area',
#                                  'color': "green",
#                                  'symbol': "circle"},))
    time.sleep(1)

IndexError: single positional indexer is out-of-bounds

In [47]:
pipe.close()

Slider
--------

In [22]:
import plotly.plotly as py
import plotly.graph_objs as go 
from pprint import *
import time, random
from datetime import datetime
# import pandas_datareader.data as web

In [17]:
import pymongo
from pymongo import MongoClient
from pprint import *
import threading

client = MongoClient('mongodb://localhost:27017/')
db = client['regular_interval']
col = db["hashtags"]

def complete_hashtag(hashtag):
    records = col.find({"hashtag":hashtag},{"timestamp":1,"count":1,"pos":1,"neg":1})
    return  list(records)
def most_distributed():
    pipeline = [{"$group" : { "_id" : '$hashtag', "count" : {"$sum" : 1}}},{"$sort" : {"count":-1}}]
    pprint(list(col.aggregate(pipeline))[:20])

In [27]:
most_distributed()

[{'_id': 'NewProfilePic', 'count': 111},
 {'_id': 'RT', 'count': 86},
 {'_id': 'BTS', 'count': 66},
 {'_id': '拡散希望', 'count': 57},
 {'_id': '방탄소년단', 'count': 53},
 {'_id': '新しいプロフィール画像', 'count': 52},
 {'_id': 'JIMIN', 'count': 51},
 {'_id': 'EXO', 'count': 48},
 {'_id': '워너원', 'count': 45},
 {'_id': 'Trump', 'count': 43},
 {'_id': 'USA', 'count': 43},
 {'_id': 'LasVegas', 'count': 42},
 {'_id': 'DNA', 'count': 42},
 {'_id': 'รูปที่เซฟไว้ดูตอนเครียด', 'count': 41},
 {'_id': 'pixiv', 'count': 40},
 {'_id': 'odaibako', 'count': 40},
 {'_id': '강다니엘', 'count': 39},
 {'_id': 'art', 'count': 39},
 {'_id': '정국', 'count': 37},
 {'_id': 'TBT', 'count': 37}]


In [28]:
l = complete_hashtag('NewProfilePic')
pprint(l)

[{'_id': ObjectId('5a6636b6e8eecb3ec1894d2e'),
  'count': 1,
  'neg': 0,
  'pos': 1,
  'timestamp': 1516648107},
 {'_id': ObjectId('5a6636cae8eecb3ec18950f1'),
  'count': 1,
  'neg': 0,
  'pos': 1,
  'timestamp': 1516648127},
 {'_id': ObjectId('5a6636d4e8eecb3ec18953b5'),
  'count': 1,
  'neg': 0,
  'pos': 1,
  'timestamp': 1516648137},
 {'_id': ObjectId('5a6636fce8eecb3ec1895dc6'),
  'count': 4,
  'neg': 0,
  'pos': 4,
  'timestamp': 1516648177},
 {'_id': ObjectId('5a663706e8eecb3ec1895f26'),
  'count': 6,
  'neg': 0,
  'pos': 6,
  'timestamp': 1516648187},
 {'_id': ObjectId('5a663710e8eecb3ec1895fea'),
  'count': 1,
  'neg': 0,
  'pos': 1,
  'timestamp': 1516648197},
 {'_id': ObjectId('5a66371ae8eecb3ec18966b0'),
  'count': 1,
  'neg': 0,
  'pos': 1,
  'timestamp': 1516648207},
 {'_id': ObjectId('5a663724e8eecb3ec1896857'),
  'count': 1,
  'neg': 0,
  'pos': 1,
  'timestamp': 1516648217},
 {'_id': ObjectId('5a66372ee8eecb3ec1896d2b'),
  'count': 2,
  'neg': 0,
  'pos': 2,
  'timestam

In [12]:
import os
os.environ['HTTP_PROXY']="http://proxy62.iitd.ac.in:3128"
os.environ['HTTPS_PROXY']="https://proxy62.iitd.ac.in:3128"

In [15]:
df = web.DataReader("aapl", 'yahoo',
                    datetime(2007, 10, 1),
                    datetime(2009, 4, 1))
pprint(df.iloc[0])

Open         2.209000e+01
High         2.248714e+01
Low          2.184714e+01
Close        2.233429e+01
Adj Close    2.002120e+01
Volume       2.092671e+08
Name: 2007-10-01 00:00:00, dtype: float64


In [16]:
pprint(df)

                 Open       High        Low      Close  Adj Close     Volume
Date                                                                        
2007-10-01  22.090000  22.487143  21.847143  22.334286  20.021204  209267100
2007-10-02  22.364286  22.655714  22.270000  22.635714  20.291412  198017400
2007-10-03  22.540001  22.740000  22.430000  22.559999  20.223536  173129600
2007-10-04  22.571428  22.582857  21.928572  22.320000  20.008396  164239600
2007-10-05  22.624287  23.082857  22.528572  23.064285  20.675600  235867800
2007-10-08  23.355715  23.987143  23.281429  23.987143  21.502874  208982200
2007-10-09  24.314285  24.444286  23.811428  23.980000  21.496475  276071600
2007-10-10  23.935715  23.982857  23.657143  23.827143  21.359447  166897500
2007-10-11  24.212856  24.554285  21.887142  23.175714  20.775484  410998000
2007-10-12  23.287144  23.897142  23.114286  23.892857  21.418356  247044000
2007-10-15  23.997143  24.224285  23.357143  23.854286  21.383780  269482500

In [30]:
t1 = int(time.time())
X = [datetime.fromtimestamp(x["timestamp"]) for x in l]
Y1 = [x["count"] for x in l]
Y2 = [x["neg"] for x in l]
trace1 = go.Scatter(x=X,y=Y1)
trace2 = go.Scatter(x=X,y=Y2)

data = [trace1,trace2]
for k in range(len(data)):
    data[k].update(
        {
            "type": "scatter",
            "hoverinfo": "name+x+text",
            "line": {"width": 0.5}, 
            "marker": {"size": 8},
            "mode": "lines+markers",
            "showlegend": False
        }
    )
layout = dict(
    title='Time series with range slider and selectors',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=15,
                     label='15m',
                     step='minute',
                     stepmode='backward'),
                dict(count=5,
                     label='5m',
                     step='minute',
                     stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(),
        type='date'
    )
)

fig = dict(data=data, layout=layout)
py.iplot(fig)

In [18]:
trace1 = go.Scatter(
     x = ["2013-01-15", "2013-01-29", "2013-02-26", "2013-04-19", "2013-07-02", "2013-08-27",
         "2013-10-22", "2014-01-20", "2014-05-05", "2014-07-01", "2015-02-09", "2015-04-13",
         "2015-05-13", "2015-06-08", "2015-08-05", "2016-02-25"], 
    y = ["8", "3", "2", "10", "5", "5", "6", "8", "3", "3", "7", "5", "10", "10", "9", "14"], 
    name = "var0", 
    text = ["8", "3", "2", "10", "5", "5", "6", "8", "3", "3", "7", "5", "10", "10", "9", "14"], 
    yaxis = "y", 
)

trace2 = go.Scatter(
    x = ["2015-04-13", "2015-05-13", "2015-06-08", "2015-08-05", "2016-02-25"], 
    y = ["53.0", "69.0", "89.0", "41.0", "41.0"], 
    name = "var1", 
    text = ["53.0", "69.0", "89.0", "41.0", "41.0"], 
    yaxis = "y2", 
)
data = go.Data([trace1, trace2])


In [19]:
# style all the traces
for k in range(len(data)):
    data[k].update(
        {
            "type": "scatter",
            "hoverinfo": "name+x+text",
            "line": {"width": 0.5}, 
            "marker": {"size": 8},
            "mode": "lines+markers",
            "showlegend": False
        }
    )


In [20]:
layout = {
  "annotations": [
    {
      "x": "2013-06-01", 
      "y": 0, 
      "arrowcolor": "rgba(63, 81, 181, 0.2)", 
      "arrowsize": 0.3, 
      "ax": 0, 
      "ay": 30, 
      "text": "state1", 
      "xref": "x", 
      "yanchor": "bottom", 
      "yref": "y"
    }, 
    {
      "x": "2014-09-13", 
      "y": 0, 
      "arrowcolor": "rgba(76, 175, 80, 0.1)", 
      "arrowsize": 0.3, 
      "ax": 0,
      "ay": 30,
      "text": "state2",
      "xref": "x", 
      "yanchor": "bottom", 
      "yref": "y"
    }
  ], 
  "dragmode": "zoom", 
  "hovermode": "x", 
  "legend": {"traceorder": "reversed"}, 
  "margin": {
    "t": 100, 
    "b": 100
  }, 
  "shapes": [
    {
      "fillcolor": "rgba(63, 81, 181, 0.2)", 
      "line": {"width": 0}, 
      "type": "rect", 
      "x0": "2013-01-15", 
      "x1": "2013-10-17", 
      "xref": "x", 
      "y0": 0, 
      "y1": 0.95, 
      "yref": "paper"
    }, 
    {
      "fillcolor": "rgba(76, 175, 80, 0.1)", 
      "line": {"width": 0}, 
      "type": "rect", 
      "x0": "2013-10-22", 
      "x1": "2015-08-05", 
      "xref": "x", 
      "y0": 0, 
      "y1": 0.95, 
      "yref": "paper"
    }
  ], 
  "xaxis": {
    "autorange": True, 
    "range": ["2012-10-31 18:36:37.3129", "2016-05-10 05:23:22.6871"], 
    "rangeslider": {
      "autorange": True, 
      "range": ["2012-10-31 18:36:37.3129", "2016-05-10 05:23:22.6871"]
    }, 
    "type": "date"
  }, 
  "yaxis": {
    "anchor": "x", 
    "autorange": True, 
    "domain": [0, 0.2], 
    "linecolor": "#673ab7", 
    "mirror": True, 
    "range": [-60.0858369099, 28.4406294707], 
    "showline": True, 
    "side": "right", 
    "tickfont": {"color": "#673ab7"}, 
    "tickmode": "auto", 
    "ticks": "", 
    "titlefont": {"color": "#673ab7"}, 
    "type": "linear", 
    "zeroline": False
  }, 
  "yaxis2": {
    "anchor": "x", 
    "autorange": True, 
    "domain": [0.2, 0.4], 
    "linecolor": "#E91E63", 
    "mirror": True, 
    "range": [29.3787777032, 100.621222297], 
    "showline": True, 
    "side": "right", 
    "tickfont": {"color": "#E91E63"}, 
    "tickmode": "auto", 
    "ticks": "", 
    "titlefont": {"color": "#E91E63"}, 
    "type": "linear", 
    "zeroline": False
  }, 
  "yaxis3": {
    "anchor": "x", 
    "autorange": True, 
    "domain": [0.4, 0.6], 
    "linecolor": "#795548", 
    "mirror": True, 
    "range": [-3.73690396239, 22.2369039624], 
    "showline": True, 
    "side": "right", 
    "tickfont": {"color": "#795548"}, 
    "tickmode": "auto", 
    "ticks": "", 
    "title": "mg/L", 
    "titlefont": {"color": "#795548"}, 
    "type": "linear", 
    "zeroline": False
  }, 
  "yaxis4": {
    "anchor": "x", 
    "autorange": True, 
    "domain": [0.6, 0.8], 
    "linecolor": "#607d8b", 
    "mirror": True, 
    "range": [6.63368032236, 8.26631967764], 
    "showline": True, 
    "side": "right", 
    "tickfont": {"color": "#607d8b"}, 
    "tickmode": "auto", 
    "ticks": "", 
    "title": "mmol/L", 
    "titlefont": {"color": "#607d8b"}, 
    "type": "linear", 
    "zeroline": False
  }, 
  "yaxis5": {
    "anchor": "x", 
    "autorange": True, 
    "domain": [0.8, 1], 
    "linecolor": "#2196F3", 
    "mirror": True, 
    "range": [-685.336803224, 3718.33680322], 
    "showline": True, 
    "side": "right", 
    "tickfont": {"color": "#2196F3"}, 
    "tickmode": "auto",
    "ticks": "", 
    "title": "mg/Kg", 
    "titlefont": {"color": "#2196F3"}, 
    "type": "linear", 
    "zeroline": False
  }
}

In [21]:
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)